In [0]:
!git clone https://github.com/zhan006/TensorFlow-Tutorials.git
import os
os.chdir("/content/TensorFlow-Tutorials/")
import tensorflow as tf
from mnist import MNIST
import mnist
import matplotlib.pyplot as plt
import numpy as np

data=MNIST()
imgshape=data.img_shape

def plotnumber(arr,label,pre_label=None):
  fig,axe=plt.subplots(3,3)
  fig.subplots_adjust(hspace=1,wspace=0.3)
  for i,ax in enumerate(axe.flat):
    ax.imshow(arr[i].reshape(imgshape))
    
    ax.set_xticks([])
    ax.set_yticks([])
    if pre_label is None:
        xlabel=label[i]
        ax.set_xlabel('true: '+str(xlabel))
    else:
        xlabel=label[i]
        tlabel=pre_label[i]
        ax.set_xlabel('true:{} pre:{}'.format(xlabel,tlabel))
  
  plt.show()


  
def training(number):
    for i in range(number):
        xbat,ybat,_=data.random_batch(50)
        #print(xbat.shape,ybat.shape)
        feeddict={x:xbat,y_:ybat,dropout:0.5}
        if i%100==0:
            print('accuracy in step{}: {}'.format(i,acce()))
        sess.run(optimizer,feed_dict=feeddict)                  
def acce():
    feed={x:x_test,y_true:y_cls,dropout:1.0}
    a=sess.run(acc,feed_dict=feed)
    print(a)
    return a
def prediction():
  feed={x:x_test,dropout:1}
  pr=sess.run(y_pred_cls,feed_dict=feed)
  return pr
def pltweights():
  w=sess.run(weights)
  fig,axe=plt.subplots(3,4)
  wmax=np.max(w)
  wmin=np.min(w)
  for i,axe in enumerate(axe.flat):
    if i<10:
      image=w[:,i].reshape(imgshape)
      axe.imshow(image,vmax=wmax,vmin=wmin,cmap='seismic')
    axe.set_xticks([])
    axe.set_yticks([])
  plt.show()  
def init_weight(shape):
  init=tf.truncated_normal(shape,stddev=0.1)
  return tf.Variable(init)
def init_bias(shape):
  init=tf.constant(0.1,shape=shape)
  return tf.Variable(init)
def conv2(x,W):
  return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")
def maxpooling(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
x=tf.placeholder('float',[None,784],name="origin")


x_image=tf.reshape(x,[-1,28,28,1])
cov1_weight=init_weight([5,5,1,32])
cov1_bias=init_bias([1,32])
cov1_c=conv2(x_image,cov1_weight)+cov1_bias
cov1_a=tf.nn.relu(cov1_c)
cov1_max=maxpooling(cov1_a)

cov2_weight=init_weight([3,3,32,64])
cov2_bias=init_bias([64])
cov2_c=conv2(cov1_max,cov2_weight)+cov2_bias
cov2_a=tf.nn.relu(cov2_c)
cov_max=maxpooling(cov2_a)

fc1_input=tf.reshape(cov_max,[-1,7*7*64])
weight1=init_weight([7*7*64,1024])
bias1=init_bias([1024])
fc1_logit=tf.matmul(fc1_input,weight1)+bias1
fc1_ac=tf.nn.relu(fc1_logit)

weight2=init_weight([1024,10])
bias2=init_bias([10])
dropout=tf.placeholder('float',name='dropout')
fc1_ac_drop=tf.nn.dropout(fc1_ac,dropout)

fc2_logit=tf.matmul(fc1_ac_drop,weight2)+bias2
y_pred=tf.nn.softmax(fc2_logit)
y_pred_cls=tf.argmax(y_pred,1)
y_=tf.placeholder('float',[None,10],name="truelabel")
loss = -tf.reduce_mean(y_ * tf.log(y_pred))
x_test=data.x_test
y_test=data.y_test
y_cls=data.y_test_cls
#print(y_cls)
sess=tf.Session()
optimizer=tf.train.GradientDescentOptimizer(0.1).minimize(loss)
#optimizer=tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.global_variables_initializer())
y_true=tf.placeholder(tf.int64,[None])
accuracy=tf.equal(y_true,y_pred_cls)
acc=tf.reduce_mean(tf.cast(accuracy,tf.float32))
training(5000)
acce()
pr=prediction()
plotnumber(x_test[5:14],y_cls[5:14],pr[5:14])
#pltweights()


fatal: destination path 'TensorFlow-Tutorials' already exists and is not an empty directory.
0.096
accuracy in step0: 0.09600000083446503
0.8388
accuracy in step100: 0.8388000130653381
0.9081
accuracy in step200: 0.9081000089645386
0.9317
accuracy in step300: 0.9316999912261963
